In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

#this network is used
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from sklearn.metrics import classification_report
import torch.nn.functional as F
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
subjects_info = pd.read_csv('/kaggle/input/open-nuro-dataset/dataset/participants.tsv', delimiter='\t')
from sklearn.model_selection import train_test_split
import pandas as pd

# First, let's identify the unique groups in the DataFrame
groups = subjects_info['Group'].unique()
groups=["A","C"]
# Now, let's split each group individually
train_dfs = []
test_dfs = []
total=[]
for group in groups:
    # Filter the DataFrame for the current group
    group_df = subjects_info[subjects_info['Group'] == group]
    
    # Split the group data into training and testing sets while maintaining balance in gender
    train_group, test_group = train_test_split(group_df, test_size=0.3, stratify=group_df['Gender'], random_state=42)
    total.append(group_df)
    # Append the split data to the lists
    train_dfs.append(train_group)
    test_dfs.append(test_group)

# Concatenate the training and testing DataFrames for all groups
train_df = pd.concat(train_dfs)
test_df = pd.concat(test_dfs)
total_df=pd.concat(total)
# Now, train_df and test_df contain the split data with balanced groups and secondary balance in gender
# Extracting subject IDs from the training set
training_subjects = train_df['participant_id'].str.extract(r'sub-(\d+)').astype(int).squeeze().unique().tolist()

# Extracting subject IDs from the testing set
testing_subjects = test_df['participant_id'].str.extract(r'sub-(\d+)').astype(int).squeeze().unique().tolist()

total_subjects = total_df['participant_id'].str.extract(r'sub-(\d+)').astype(int).squeeze().unique().tolist()

# Displaying the lists of subjects
print("Training Subjects:")
print(training_subjects)

print("\nTesting Subjects:")
print(testing_subjects)

print("\nTotal Subjects:")
print(total_subjects)

In [ ]:
##laods the data

import numpy as np
from scipy.io import loadmat
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def load_data(subjects_info,subjects, data_type='training'):
    groups=[]
    coherences = []
    rbps = []
    scc=[]
    labels = []
    for idx in subjects:
        output_folder="/kaggle/input/fork-of-extraction-cleaned"
        
        file_path_rbp =  os.path.join(output_folder, f'rbp/rbp_{idx}.npy')
        file_path_coherence =  os.path.join(output_folder, f'coherence/coherence_{idx}.npy')
        file_path_scc =  os.path.join(output_folder, f'scc_cleaned_base/sub-{idx}_epochs.npy')
        
        
        
        subject_data_coherence = np.load(file_path_coherence)
        
        coherences.append(subject_data_coherence)
        

        subject_rbp = np.load(file_path_rbp)
        
        rbps.append(subject_rbp)
        
        subject_scc = np.load(file_path_scc)
        
        scc.append(subject_scc)
        
        # Duplicate entries in the 'Group' column
        num_epochs = subject_rbp.shape[0]  # Number of epochs, replace 'your_data_key' as before
        subject_id = f"sub-{str(idx).zfill(3)}"  # Format subject ID
        group_info = subjects_info[subjects_info['participant_id'] == subject_id]['Group'].values[0]
        duplicated_groups = [group_info] * num_epochs
        labels.extend(duplicated_groups)
        
        groups.extend([idx] * num_epochs)
        
    return np.concatenate(coherences, axis=0), np.concatenate(rbps, axis=0),np.concatenate(scc, axis=0),labels,groups

In [ ]:
###source file
subjects_info = pd.read_csv('/kaggle/input/open-nuro-dataset/dataset/participants.tsv', delimiter='\t')
##########

In [ ]:
coherences_total, rbps_total, scc_total, total_labels, groups_total = load_data(subjects_info, total_subjects, data_type='total')

In [ ]:
coherences_total.shape

In [ ]:
# Assuming you define label_mapping, encoder, and encode labels as in your code
#####################################################
label_mapping = {'A': 0, 'C': 1, 'F': 2}  # You can adjust this mapping as needed
#####################################################
df=pd.Series(total_labels).map(label_mapping)

encoder = OneHotEncoder(categories='auto', sparse=False)
encoder.fit(np.array(pd.Series(total_labels).map(label_mapping)).reshape(-1, 1))
# Encode labels
################################################

total_labels_encoded = encoder.transform(np.array(pd.Series(total_labels).map(label_mapping).values).reshape(-1, 1))
################################################


print("Total labels shape:", total_labels_encoded.shape)

In [ ]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
#####total dataset 
coherence_tensor_total = torch.tensor(scc_total, dtype=torch.float32).unsqueeze(1)
rbps_tensor_total = torch.tensor(rbps_total, dtype=torch.float32).unsqueeze(1)
labels_tensor_total = torch.tensor(total_labels_encoded, dtype=torch.float32)

dataset_total = TensorDataset(coherence_tensor_total,rbps_tensor_total, labels_tensor_total)

batch_size = 64

train_loader = DataLoader(dataset_total, batch_size=batch_size, shuffle=True,drop_last=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction_ratio=8):
        super(SEBlock, self).__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction_ratio),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction_ratio, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        batch_size, channels, _, _ = x.size()
        out = self.squeeze(x).view(batch_size, channels)
        out = self.excitation(out).view(batch_size, channels, 1, 1)
        return x * out.expand_as(x)

class ConvNet(nn.Module):
    def __init__(self, dropout_rate=0.15, num_classes=2):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.se1 = SEBlock(64)  # SE block after conv1
        self.conv2 = nn.Conv2d(64, 16, kernel_size=3, padding=1)
        self.se2 = SEBlock(16)  # SE block after conv2
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 42, 128)
        self.dropout1 = nn.Dropout(dropout_rate)

        # Transformer layers
        self.transformer_layer1 = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder1 = nn.TransformerEncoder(self.transformer_layer1, num_layers=2)
        self.dropout2 = nn.Dropout(dropout_rate)

        # Second input path
        self.conv3 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.se3 = SEBlock(32)  # SE block after conv3
        self.conv4 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.se4 = SEBlock(16)  # SE block after conv4
        self.fc3 = nn.Linear(16 * 4, 128)
        self.dropout3 = nn.Dropout(dropout_rate)

        # Transformer layers for the second input path
        self.transformer_layer2 = nn.TransformerEncoderLayer(d_model=128, nhead=8)
        self.transformer_encoder2 = nn.TransformerEncoder(self.transformer_layer2, num_layers=1)
        self.dropout4 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, num_classes)  # Change to 3 for 3 classes

    def forward(self, x1, x2):
        x1 = self.pool(F.relu(self.se1(self.conv1(x1))))
        x1 = self.pool(F.relu(self.se2(self.conv2(x1))))
        x1 = x1.view(-1, 16 * 42)
        x1 = F.relu(self.fc1(x1))
        x1 = self.dropout1(x1)

        # Transformer layers
        seq_len = x1.size(0)
        pos_encoding = self.get_positional_encoding(seq_len, 128).to(x1.device)
        x1 = x1 + pos_encoding
        x1 = x1.unsqueeze(1)
        x1 = self.transformer_encoder1(x1)
        x1 = self.dropout2(x1.squeeze(1))

        x2 = self.pool(F.relu(self.se3(self.conv3(x2))))
        x2 = self.pool(F.relu(self.se4(self.conv4(x2))))
        x2 = x2.view(-1, 16 * 4)
        x2 = F.relu(self.fc3(x2))
        x2 = self.dropout3(x2)

        # Transformer layers for the second input path
        seq_len = x2.size(0)
        pos_encoding = self.get_positional_encoding(seq_len, 128).to(x2.device)
        x2 = x2 + pos_encoding
        x2 = x2.unsqueeze(1)
        x2 = self.transformer_encoder2(x2)
        x2 = self.dropout4(x2.squeeze(1))

        x = torch.cat((x1, x2), dim=1)
        x = F.relu(self.fc2(x))
        x = self.fc4(x)
        return x

    def get_positional_encoding(self, seq_len, d_model):
        pos_encoding = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len).float().unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        return pos_encoding

In [ ]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold ,LeaveOneGroupOut
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Assuming your dataset and groups are already defined
# dataset_total = ...
# groups_total = ...
# groups_grouping = ...  # Define group labels for StratifiedGroupKFold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
inner_skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

sgkf = StratifiedGroupKFold(n_splits=10, shuffle=True, random_state=42)

binary_labels = np.any(total_labels_encoded, axis=1).astype(int)  # Convert to binary (0 or 1)

group_indices = skf.split(np.arange(len(dataset_total)), groups_total)


#group_indices = sgkf.split(np.arange(len(dataset_total)), binary_labels, groups_total)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 256

fold = 1
#group_indices = skf.split(np.arange(len(dataset_total)), groups_total, groups_grouping)
total_y_true = []
total_y_pred = []

train_losses = []
val_losses = []

# List to store metrics for each fold
fold_metrics = []

for train_indices, test_indices in group_indices:
    print(f"Fold {fold}")
    
    # Split the dataset into train and test sets based on indices
    train_data = [dataset_total[i] for i in train_indices]
    test_data = [dataset_total[i] for i in test_indices]
    
    # Split the training data into training and validation sets
    train_split_indices, val_split_indices = next(inner_skf.split(np.arange(len(train_data)), [groups_total[i] for i in train_indices]))
    train_subset = [train_data[i] for i in train_split_indices]
    val_subset = [train_data[i] for i in val_split_indices]
    
    # Create DataLoader for train, validation, and test sets
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    
    # Initialize the model and optimizer
    model = ConvNet().to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Early stopping parameters
    patience = 25  # Number of epochs to wait before early stopping
    best_val_loss = float('inf')
    early_stop_counter = 0
    best_model_state = None  # Variable to store the best model's state
    
    # Training loop
    num_epochs = 60
    train_loss_list = []
    val_loss_list = []
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        
        model.train()
        for inputs1, inputs2, labels in train_loader:
            optimizer.zero_grad()
            inputs1 = inputs1.to(device)
            inputs2 = inputs2.to(device)
            labels = labels.to(device)
            outputs = model(inputs1, inputs2)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        # Calculate validation loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs1, inputs2, labels in val_loader:
                inputs1 = inputs1.to(device)
                inputs2 = inputs2.to(device)
                labels = labels.to(device)
                outputs = model(inputs1, inputs2)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss}")
        
        train_loss_list.append(running_loss/len(train_loader))
        val_loss_list.append(val_loss)
        
        # Early stopping and save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_state = model.state_dict()  # Save the best model's state
            early_stop_counter = 0
        else:
            early_stop_counter += 1
        
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Load the best model's state
    model.load_state_dict(best_model_state)
    
    # Evaluate on the test set
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for inputs1, inputs2, labels in test_loader:
            inputs1 = inputs1.to(device)
            inputs2 = inputs2.to(device)
            labels = labels.to(device)
            outputs = model(inputs1, inputs2)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().tolist())
            y_pred.extend(predicted.cpu().tolist())
            total_y_true.extend(labels.cpu().tolist())
            total_y_pred.extend(predicted.cpu().tolist())
    
    # Calculate classification metrics
    y_true = np.argmax(y_true, axis=1)
    acc = accuracy_score(y_true, y_pred)
    sens = recall_score(y_true, y_pred, average='macro', pos_label=1)
    spec = recall_score(y_true, y_pred, average='macro', pos_label=0)
    prec = precision_score(y_true, y_pred, average='macro', pos_label=1)
    f1 = f1_score(y_true, y_pred, average='macro', pos_label=1)
    recall = recall_score(y_true, y_pred, average='macro', pos_label=1)
    confusion_mat = confusion_matrix(y_true, y_pred)
    
    # Store metrics for the current fold
    fold_metrics.append({
        "Fold": fold,
        "Accuracy": acc,
        "Sensitivity": sens,
        "Specificity": spec,
        "Precision": prec,
        "F1 Score": f1,
        "Recall": recall,
        "Confusion Matrix": confusion_mat
    })
    
    print(f"Fold {fold}/10 Classification Metrics:")
    print(f"Accuracy: {acc:.4f}")
    print(f"Sensitivity: {sens:.4f}")
    print(f"Specificity: {spec:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Confusion Matrix:\n{confusion_mat}")
    print("\n")
    
    # Plot training curves for the current fold
    plt.figure()
    plt.plot(range(len(train_loss_list)), train_loss_list, label='Train Loss')
    plt.plot(range(len(val_loss_list)), val_loss_list, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title(f'Training Curve for Fold {fold}')
    plt.legend()
    plt.show()
    
    train_losses.extend(train_loss_list[:epoch + 1])  # Save up to the epoch where early stopping occurred
    val_losses.extend(val_loss_list[:epoch + 1])      # Save up to the epoch where early stopping occurred
    
    fold += 1

# Calculate overall classification metrics
total_y_true = np.argmax(total_y_true, axis=1)
acc = accuracy_score(total_y_true, total_y_pred)
sens = recall_score(total_y_true, total_y_pred, average='macro', pos_label=1)
spec = recall_score(total_y_true, total_y_pred, average='macro', pos_label=0)
prec = precision_score(total_y_true, total_y_pred, average='macro', pos_label=1)
f1 = f1_score(total_y_true, total_y_pred, average='macro', pos_label=1)
recall = recall_score(total_y_true, total_y_pred, average='macro', pos_label=1)
confusion_mat = confusion_matrix(total_y_true, total_y_pred)

print("Overall Classification Metrics:")
print(f"Accuracy: {acc:.4f}")
print(f"Sensitivity: {sens:.4f}")
print(f"Specificity: {spec:.4f}")
print(f"Precision: {prec:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Confusion Matrix:\n{confusion_mat}")
print("\n")


report = classification_report(total_y_true, total_y_pred)
print("Total Classification Report:")
print(report)
print("\n")

import numpy as np

# Extract metrics into arrays for standard deviation calculation
accuracies = [m['Accuracy'] for m in fold_metrics]
sensitivities = [m['Sensitivity'] for m in fold_metrics]
specificities = [m['Specificity'] for m in fold_metrics]
precisions = [m['Precision'] for m in fold_metrics]
f1_scores = [m['F1 Score'] for m in fold_metrics]
recalls = [m['Recall'] for m in fold_metrics]

# Calculate means
accuracy_mean = np.mean(accuracies)
sensitivity_mean = np.mean(sensitivities)
specificity_mean = np.mean(specificities)
precision_mean = np.mean(precisions)
f1_score_mean = np.mean(f1_scores)
recall_mean = np.mean(recalls)

# Calculate standard deviations
accuracy_std = np.std(accuracies)
sensitivity_std = np.std(sensitivities)
specificity_std = np.std(specificities)
precision_std = np.std(precisions)
f1_score_std = np.std(f1_scores)
recall_std = np.std(recalls)

# Print metrics for each fold
print("Metrics for each fold:")
print(f"{'Fold':<5} {'Accuracy':<10} {'Sensitivity':<12} {'Specificity':<12} {'Precision':<10} {'F1 Score':<10} {'Recall':<10}")
for metrics in fold_metrics:
    print(f"{metrics['Fold']:<5} {metrics['Accuracy']:<10.4f} {metrics['Sensitivity']:<12.4f} {metrics['Specificity']:<12.4f} {metrics['Precision']:<10.4f} {metrics['F1 Score']:<10.4f} {metrics['Recall']:<10.4f}")

# Print means and standard deviations
print("\nAverage Metrics with Standard Deviations:")
print(f"{'Metric':<12} {'Mean ± STD':<20}")
print(f"{'Accuracy':<12} {accuracy_mean:.4f} ± {accuracy_std:.4f}")
print(f"{'Sensitivity':<12} {sensitivity_mean:.4f} ± {sensitivity_std:.4f}")
print(f"{'Specificity':<12} {specificity_mean:.4f} ± {specificity_std:.4f}")
print(f"{'Precision':<12} {precision_mean:.4f} ± {precision_std:.4f}")
print(f"{'F1 Score':<12} {f1_score_mean:.4f} ± {f1_score_std:.4f}")
print(f"{'Recall':<12} {recall_mean:.4f} ± {recall_std:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate overall confusion matrix
confusion_mat = confusion_matrix(total_y_true, total_y_pred)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(6, 6))

# Plot the confusion matrix using seaborn
sns.heatmap(confusion_mat, annot=True, cmap="Blues", fmt="d", xticklabels=['Alzheimer\'s', 'Control'], yticklabels=['Alzheimer\'s', 'Control'], ax=ax)

# Set the title and labels
ax.set_title("Confusion Matrix")
ax.set_xlabel("Predicted Labels")
ax.set_ylabel("True Labels")

# Show the plot
plt.show()